<img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px;" />

# 2.3 Data Access
This is a basic introduction on how to access EO data on a cloud provided using an Application Protocol Interface (API).

The main concepts that will presented here will use openEO as API, and the openEO Python Client library.

We will interact with cloud providers offering free and open data through STAC catalogs.

This functionality allows to create openEO workflows using the same syntax as when working connected to an openEO back-end, but works with local computing resources.

The main concepts presented in this notebook are:
- Lazy data loading
- Filter operators
- Reduce operators
- Apply operators
- Aggregate operators
- Create a simple EO workflow: load, filter, apply a function, extract information


In [1]:
import openeo
from openeo.local import LocalConnection
local_conn = LocalConnection('')

C:\Users\mclaus\Miniconda3\envs\sap\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


## Lazy data loading

When accessing data using an API, most of the time the data is **lazily** loaded.

It means that just the metadata is loaded, so that it is possible to know about the data dimensions and their extents (spatial and temporal), the available bands and other additional information.

Let's start with a call to the openEO process `load_stac` for lazily loading some Sentinel-2 data from a public STAC Collection.

We need to specify an Area Of Interest (AOI) to get only part of the Collection, otherwise our code would try to load the metadata of all Sentinel-2 tiles available in the world!

In [2]:
url = "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs"
spatial_extent = {"west": 11.1, "east": 11.5, "south": 46.1, "north": 46.5}

datacube = local_conn.load_stac(url=url,
                    spatial_extent=spatial_extent)
datacube

C:\Users\mclaus\Documents\GitHub\openeo-python-client\openeo\metadata.py:255: UserWarning: No cube:dimensions metadata
  complain("No cube:dimensions metadata")


Calling the `.execute()` method, the data will be lazily loaded and an `xArray` object returned.

Running the next cell will show the selected data content with the dimension names and their extent:

In [3]:
datacube.execute()

Deserialised process graph into nested structure
Running process load_stac
kwargs: {'spatial_extent': <class 'openeo_pg_parser_networkx.pg_schema.BoundingBox'>, 'url': <class 'str'>}
--------------------------------------------------------------------------------


Walking node root-08978391-9e75-467b-a321-2e545469e2ce


<xarray.DataArray 'stackstac-02db84de21884022174303dc71ef7076' (time: 751,
                                                                band: 17,
                                                                y: 4534, x: 3209)>
dask.array<getitem, shape=(751, 17, 4534, 3209), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/29)
  * time                          (time) datetime64[ns] 2016-11-05T10:12:57 ....
    id                            (time) <U24 'S2A_32TPS_20161105_0_L2A' ... ...
  * band                          (band) <U8 'overview' 'B11' ... 'visual' 'SCL'
  * x                             (x) float64 6.611e+05 6.612e+05 ... 6.932e+05
  * y                             (y) float64 5.153e+06 5.153e+06 ... 5.107e+06
    sentinel:product_id           (time) <U60 'S2A_MSIL2A_20161105T101212_N00...
    ...                            ...
    sentinel:boa_offset_applied   (time) object None None None ... True True
    title                         (band) <U31 'True color image' ... 'Scene C...
    common_name                   (band) object None 'swir16' ... None None
    center_wavelength             (band) object None 1.6137 0.4439 ... None None
    full_width_half_max           (band) object None 0.143 0.027 ... None None
    epsg                          int32 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0

From the output of the previous cell you can notice something really interesting: **the size of the selected data is 1.35 TB!**

But you should have noticed that it was too quick to download this big amount of data.

This is what lazy loading allows: getting all the information about the data in a quick manner without having to access and download all the available files.